In [1]:
import requests
import pandas as pd
import time


In [ ]:
"""
I'm defining a function to send requests with the related parameters.
I will use il_plakası, page and page_size
page_size defines number of records to return
page means number of records in each page 
for example if I set page=1 and page_size=10 I will get the first 10 record. But if I set page = 2 and page_size = 10
then i will get the records from 11 to 20
"""

def make_request(il_plakası,page, page_size):
    url = "https://derbis.dernekler.gov.tr/IstatistikDerneklerWeb/GetIlFaaliyetDernek"
    
    serializedData = f'{{"secilenTeskilatPk":"{il_plakası}","secilenIlcePk":"999999999","neviler":"0","altneviler":"0"}}'
    payload = f"sort=&group=&filter=&page={page}&pageSize={page_size}&serializedData={serializedData}"

    headers = {
        "cookie": "ASP.NET_SessionId=fdle2s30xtn1wnux0y3dkklw",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://derbis.dernekler.gov.tr",
        "Referer": "https://derbis.dernekler.gov.tr/IstatistikDerneklerWeb/IlFaaliyetAlaniDernekler",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "sec-ch-ua": "\"Google Chrome\";v=\"119\", \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\""
    }
    
    response = requests.post(url, data=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Request failed", "status_code": response.status_code}

In [ ]:
# First I will loop through each city and get the number of NGO's
# For this I will just set the page_size = 0
results_dict = {}


for il_plakası in range(1, 82):
    result = make_request(il_plakası,1, 0)
    total_value = result.get("Total", None)  
    results_dict[il_plakası] = total_value  



In [ ]:
# This will be my main dataframe
combined_dataframe = pd.DataFrame()

# I will loop through my results dictionary. This dictionary contains provincial license plate number as key and number of NGO's as value.
for il_plakası, total_records in results_dict.items():
    if total_records < 1000:
        result = make_request(il_plakası, 1, total_records)
        
        # Check if the "Data" key exists in the result
        # This Data key is the main key for the API output.
        if "Data" in result:
            # I will convert JSON to df
            df = pd.json_normalize(result['Data'])
            
            # Add the dataframe to the main dataframe
            combined_dataframe = pd.concat([combined_dataframe, df], ignore_index=True)
        else:
            print(f"No 'Data' key in the response for il_plakası={il_plakası}")
    else:
        # As I mentioned earlier, if we have more than 1000 NGO's we need to make multiple calls.
        num_pages = math.ceil(total_records / 1000)
        for page_number in range(1, num_pages + 1):
            result = make_request(il_plakası, page_number, 1000)
            if "Data" in result:
                df = pd.json_normalize(result['Data'])
                combined_dataframe = pd.concat([combined_dataframe, df], ignore_index=True)
            else:
                print(f"No 'Data' key in the response for il_plakası={il_plakası}")

# Print the combined dataframe
print(combined_dataframe)

In [ ]:
# Export the main dataframe
combined_dataframe.to_excel("./Boran_Göksel_Assignment2_Scrapped_Data.xlsx")

In [12]:
path = 'C:/Users/boran/Desktop/Güz Dönemi/Advanced Python/Scraping Task/Boran_Göksel_Assignment2_Scrapped_Data.xlsx'
df = pd.read_excel(path)

df.drop(['Unnamed: 0', 'Ad', 'Sayi', 'Oran', 'telNo', 'Mail', 'FkSisKodPkNevi', 'FKSisTeskilatPk', 'SisKod', 'Sira', 'FkSisCityInfoPk', 'AltNevis', 'IsActive', 'DtOlusturma', 'DtDegistirme', 'Olusturan', 'Degistiren'], axis=1, inplace=True)


In [13]:
df = df[df['Plaka'] == 34]
df.reset_index(inplace=True)


In [8]:
df.drop('level_0',axis=1,inplace=True)
df.head()

,Plaka,AnaNevisi,WebSitesi,KurumAd,KurumAdres,BaskanAdSoyad,AltNevisString
0,34,İNSANİ YARDIM DERNEKLERİ,NaN,ALİYA DERNEĞİ,HOBYAR MAH. CEMAL NADİR-SOKAK DISKAPINO:24 --B...,AHMED SEMİH ÖZMEKİK,İhtiyaç Sahiplerine Yardım
1,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,(GIRASORİLİLER) KIRMIZIGÜLLER SOSYAL YARDIMLAŞ...,- AŞAĞI DUDULLU OSMANLI CADDESİ - 146 -,SANCAR YÜCE,Belirli Bir alanla ilgili Dayanışma Dernekleri
2,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,www.haycar.com,(HAYCAR) HAYRAT CANLANDIRAN VE ARAŞTIRAN MİMAR...,HALASKARGAZİ SAMANYOLU 74 4,NAZAR ŞİGAHER,"İmar, Şehircilik Ve Kalkındırma- İhtisas Mesle..."
3,34,SPOR ve SPOR İLE İLGİLİ DERNEKLERİ,www.keftud.org.tr,(KEFTUD) KEMALİYE (EĞİN) FOLKLOR- TURİZM-GENÇL...,İSKENDERPAŞA MAH. DOLAP-CADDE DISKAPINO:1 --ER...,MURAT AKOĞLU,Folklor - Turizm- Spor (Spor kulüpleri ve üst...
4,34,İNSANİ YARDIM DERNEKLERİ,NaN,1 ÇOCUK 1000 UMUT DERNEĞİ,KÜÇÜKBAKKALKÖY MAH. KAYIŞDAĞI CD. TAHSİN ARICI...,BURCUHAN YILDIRAN KUROĞLU,İhtiyaç Sahiplerine Yardım


In [14]:
# This code works on multi-thread
%%time
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_geocode_data(address, api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {'address': address, 'key': api_key}

    try:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            return None, f"HTTP Error: {response.status_code}"

        data = response.json()
        if data['status'] != 'OK' or not data['results']:
            return None, f"API Error: {data.get('status', 'No status')}"

        return data['results'][0], None
    except Exception as e:
        return None, f"Exception: {str(e)}"

def process_row(row):
    result, error = get_geocode_data(row['KurumAdres'], 'api_key')
    if error:
        return row.name, (None, None, None, None, None, None, error)

    geolocation = result['geometry']['location']
    city, district, neighborhood, partial_match = None, None, None, result.get('partial_match', False)
    for component in result['address_components']:
        if 'administrative_area_level_1' in component['types']:
            city = component['long_name']
        elif 'administrative_area_level_2' in component['types']:
            district = component['long_name']
        elif 'administrative_area_level_4' in component['types']:
            neighborhood = component['long_name']

    return row.name, (geolocation['lat'], geolocation['lng'], city, district, neighborhood, partial_match, None)

new_df = df  # Adjust as needed

# Add an Error column to your DataFrame
new_df['Error'] = None

with ThreadPoolExecutor(max_workers=14) as executor:  # Adjust the number of workers as needed
    futures = [executor.submit(process_row, row) for index, row in new_df.iterrows()]
    for future in as_completed(futures):
        index, result = future.result()
        if result[-1]:  # Checking if there's an error message
            new_df.at[index, 'Error'] = result[-1]
        else:
            # Assign each element of the result tuple to the respective column
            new_df.at[index, 'Latitude'] = result[0]
            new_df.at[index, 'Longitude'] = result[1]
            new_df.at[index, 'City'] = result[2]
            new_df.at[index, 'District'] = result[3]
            new_df.at[index, 'Neighborhood'] = result[4]
            new_df.at[index, 'PartialMatch'] = result[5]     
        
# Now new_df has the new columns with the geocoded data, 'PartialMatch' field, and 'Error' column


CPU times: total: 1h 32min 57s
Wall time: 23min 36s


In [15]:
new_df.to_excel('C:/Users/boran/Desktop/Güz Dönemi/Advanced Python/Scraping Task/geotagged.xlsx')

In [17]:
new_df.tail(30)

,index,Plaka,AnaNevisi,WebSitesi,KurumAd,KurumAdres,BaskanAdSoyad,AltNevisString,Error,Latitude,Longitude,City,District,Neighborhood,PartialMatch
23089,62847,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK GÖKÇEBEY İLÇESİ HACI MUSA VE HERKİME...,İSMET PAŞA MAH. MAH. 143 SOK. CD. NO: 26 / 5 S...,BİROL AYVACIK,Hemşehri Dernekleri,None,41.106368,28.856793,İstanbul,Sultangazi,İsmetpaşa,False
23090,62848,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK İLİ DEVREK İLÇESİ AKÇABEY VE CİVAR K...,MİMAR SİNAN MAH. 79.SOK. NO.37/A,ABDULLAH DEDELİ,Hemşehri Dernekleri,API Error: ZERO_RESULTS,NaN,NaN,NaN,NaN,NaN,NaN
23091,62849,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK İLİ DEVREK İLÇESİ DURUPINAR KÖYÜ YAR...,HADIMKÖY MAH. AHİ EVRAN SK. NO: 1/4 ARNAVUTKÖY...,RECEP ALATAŞ,Hemşehri Dernekleri,None,41.155558,28.624863,İstanbul,Arnavutköy,Hadımköy,False
23092,62850,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK İLİ DEVREK İLÇESİ EĞERCİLİLER YARDIM...,YILDIZTABYA CD. NO: 104 / A GAZİOSMANPAŞA/İSTA...,DURSUN KARTAL,Hemşehri Dernekleri,None,41.064085,28.930734,İstanbul,Gaziosmanpaşa,Yıldıztabya,False
23093,62851,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK İLİ DEVREK İLÇESİ PURTULOĞLU KÖYÜ YA...,SANAYİ TAŞÇI 18 2,ÖNDER OĞUZ,Hemşehri Dernekleri,API Error: ZERO_RESULTS,NaN,NaN,NaN,NaN,NaN,NaN
23094,62852,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK İLİ DEVREK İLÇESİ SABUNLAR KÖYÜ YARD...,TUNA MAH. SAKARYA CAD. NO:75/A,ŞEREF CAVLAK,Hemşehri Dernekleri,None,40.773074,30.394817,Sakarya,None,None,True
23095,62853,34,"KÜLTÜR, SANAT ve TURİZM DERNEKLERİ",NaN,ZONGULDAK İLİ DEVREK İLÇESİ SİPAHİLER KÖYÜ SOS...,KAZIM KARABEKİR MAH. 1016.-SOKAK DISKAPINO:4 -...,YILMAZ ÇÖPCÜ,Toplumsal Değerleri Yaşatma Dernekleri,None,41.079413,28.853855,İstanbul,Esenler,None,True
23096,62854,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK İLİ DEVREK İLÇESİ SOSYAL KÜLTÜREL DA...,FEVZİ ÇAKMAK MAH. İNÖNÜ CAD. ERTUĞRULGAZİ SK. ...,MURAT TAŞCI,Hemşehri Dernekleri,None,40.876501,29.267019,İstanbul,Pendik,Fevzi Çakmak,True
23097,62855,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK İLİ DEVREK İLÇESİ YAĞMURCA KÖYÜ YARD...,ABDURRAHMANGAZİ MAH. EBUBEKİR-CADDE DISKAPINO:...,NAMIK ŞALLI,Belirli Bir alanla ilgili Dayanışma Dernekleri,None,40.995166,29.232393,İstanbul,Sancaktepe,Abdurrahmangazi,True
23098,62856,34,MESLEKİ ve DAYANIŞMA DERNEKLERİ,NaN,ZONGULDAK KARAELMAS DİYARI SOSYAL YARDIMLAŞMA ...,İNÖNÜ MAH. KEMALPAŞA CD. NO:29/B ESENYURT/İSTA...,VOLKAN ÜNAL,Belirli Bir alanla ilgili Dayanışma Dernekleri,None,41.023628,28.691163,İstanbul,Esenyurt,İnönü,False
